# ETL Process

## **Bronze data**

In this step data is extracted from the source and saved as parquet file in the input directory. 

**(In AirFlow data will be saved at S3 bucket MinIO directory at folder *bronze/*)**

### Download

In [69]:
import os
from dotenv import load_dotenv
from massive import RESTClient

load_dotenv()
api_key = os.getenv("MASSIVE_API_KEY")

client = RESTClient(api_key=api_key)

aggs = []
for a in client.list_aggs(
    "X:BTCUSD",
    1,
    "day",
    "2023-11-27",
    "2025-11-27",
    adjusted="true",
    sort="asc",
    limit=50000,
):
    aggs.append(a)

print(aggs)

[Agg(open=37864.79, high=38152.18, low=37490, close=37732.27, volume=10547.50291791071, vwap=37763.3907, timestamp=1701302400000, transactions=235665, otc=None), Agg(open=37732.27, high=39000, low=37618, close=38703.54, volume=22789.851736040953, vwap=38527.3418, timestamp=1701388800000, transactions=405390, otc=None), Agg(open=38690, high=39740, low=38634, close=39478.95, volume=12361.73849266264, vwap=39151.5397, timestamp=1701475200000, transactions=276518, otc=None), Agg(open=39467.42, high=40221.22, low=39253, close=39984.81, volume=10699.749838323205, vwap=39704.2976, timestamp=1701561600000, transactions=280060, otc=None), Agg(open=39975.1, high=42415.5, low=39476, close=41987.24, volume=32400.89486495723, vwap=41409.4421, timestamp=1701648000000, transactions=565834, otc=None), Agg(open=41983, high=45000, low=40849, close=44084.39, volume=34916.9349298997, vwap=42953.9916, timestamp=1701734400000, transactions=591912, otc=None), Agg(open=44086.33, high=44297, low=43335.5, close

### Load data to dataframe

In [70]:
import pandas as pd

data = pd.DataFrame(aggs)


## **Silver data**

Here data is cleared and indexed by timestamp

In [71]:
data["timestamp"] = pd.to_datetime(data["timestamp"], unit="ms").dt.tz_localize(None)
data = data.rename(
    columns={
        "timestamp": "date"
    }
)
data = data.set_index("date")

data = data.drop(
    columns=[
        "otc", 
        "transactions"
    ]
)


print(data)

                open      high       low     close        volume        vwap
date                                                                        
2023-11-30  37864.79  38152.18  37490.00  37732.27  10547.502918  37763.3907
2023-12-01  37732.27  39000.00  37618.00  38703.54  22789.851736  38527.3418
2023-12-02  38690.00  39740.00  38634.00  39478.95  12361.738493  39151.5397
2023-12-03  39467.42  40221.22  39253.00  39984.81  10699.749838  39704.2976
2023-12-04  39975.10  42415.50  39476.00  41987.24  32400.894865  41409.4421
...              ...       ...       ...       ...           ...         ...
2025-11-23  84679.90  88290.00  84614.11  86808.28  13247.961570  86625.3368
2025-11-24  86777.50  89349.00  85250.00  88275.00   3651.817757  87316.0245
2025-11-25  88275.00  88628.00  86100.00  87326.30   2526.067553  87393.6304
2025-11-26  87326.40  90684.00  86299.50  90535.70   2964.592284  88479.9401
2025-11-27  90535.70  91990.00  90067.81  91316.70  10126.004871  91198.3559

## **Gold data**

Here data is processed by alpha158 and alpha360

In [72]:
def calculate_alpha360(df):
    df = df.copy()
    for i in range(59, 0, -1):
        df[f"close{i}"] = df["close"].shift(i)/df["close"]
        df[f"high{i}"] = df["high"].shift(i)/df["close"]
        df[f"low{i}"] = df["low"].shift(i)/df["close"]
        df[f"open{i}"] = df["open"].shift(i)/df["close"]
        df[f"volume{i}"] = df["volume"].shift(i)/(df["volume"]+1e-12)
        df[f"vwap{i}"] = df["vwap"].shift(i)/df["close"]
    return df

### **1. KMID – Candle Body Relative to Open**

**Definicja:**
Miara wielkości korpusu świecy względem ceny otwarcia. Informuje, jak duża była zmiana ceny w stosunku do tego, gdzie rynek zaczął.

**Wzór:**
$$
\text{KMID} = \frac{close - open}{open}
$$


### **2. KLEN – Candle Range Relative to Open**

**Definicja:**
Pokazuje całkowity zakres świecy (od minimum do maksimum) w relacji do ceny otwarcia. Mierzy ogólną zmienność świecy.

**Wzór:**
$$
\text{KLEN} = \frac{high - low}{open}
$$


### **3. KMID2 – Candle Body Relative to Total Range**

**Definicja:**
Proporcja korpusu świecy (close–open) do całkowitego zakresu (high–low). Mierzy, jak duża część całej zmienności została „wypełniona” ruchem ceny netto.

**Wzór:**
$$
\text{KMID2} = \frac{close - open}{high - low + 10^{-12}}
$$


### **4. KUP – Upper Shadow Relative to Open**

**Definicja:**
Długość górnego knota świecy mierzona względem ceny otwarcia. Używa maksimum oraz wyższego z open/close.

**Wzór:**
$$
\text{KUP} = \frac{high - \max(open, close)}{open}
$$


### **5. KUP2 – Upper Shadow Relative to Total Range**

**Definicja:**
Długość górnego cienia w stosunku do pełnego zakresu świecy.

**Wzór:**
$$
\text{KUP2} = \frac{high - \max(open, close)}{high - low + 10^{-12}}
$$


### **6. KLOW – Lower Shadow Relative to Open**

**Definicja:**
Długość dolnego cienia świecy mierzona względem ceny otwarcia. Wykorzystuje minimum oraz niższy z open/close.

**Wzór:**
$$
\text{KLOW} = \frac{\min(open, close) - low}{open}
$$


### **7. KLOW2 – Lower Shadow Relative to Total Range**

**Definicja:**
Proporcja dolnego knota świecy względem całkowitego zakresu świecy.

**Wzór:**
$$
\text{KLOW2} = \frac{\min(open, close) - low}{high - low + 10^{-12}}
$$


### **8. KSFT – Candle Shift Relative to Open**

**Definicja:**
Miara przesunięcia ceny zamknięcia wewnątrz świecy względem open — pokazuje „pozycję” close między skrajnościami high i low.

Wzór pochodzi z transformacji:
$$
2 \cdot close - high - low
$$
co wskazuje przesunięcie close od środka świecy.

**Wzór:**
$$
\text{KSFT} = \frac{2 \cdot close - high - low}{open}
$$


### **9. KSFT2 – Candle Shift Relative to Total Range**

**Definicja:**
Ta sama idea co KSFT, ale odniesiona do całkowitego zakresu zmienności świecy.

**Wzór:**
$$
\text{KSFT2} = \frac{2 \cdot close - high - low}{high - low + 10^{-12}}
$$


### **10. ROC(d) – Rate of Change**

**Definicja:**
Zmiana ceny zamknięcia sprzed *d* dni względem bieżącej ceny zamknięcia. Normalizowana przez aktualne close.

**Wzór:**
$$
ROC_d = \frac{Ref(close, d)}{close}
$$


### **11. MA(d) – Simple Moving Average (normalized)**

**Definicja:**
Średnia krocząca ceny zamknięcia z ostatnich *d* dni, podzielona przez bieżące close.

**Wzór:**
$$
MA_d = \frac{Mean(close, d)}{close}
$$


### **12. STD(d) – Standard Deviation of Close (normalized)**

**Definicja:**
Odchylenie standardowe ceny zamknięcia z ostatnich *d* dni, znormalizowane przez bieżące close.

**Wzór:**
$$
STD_d = \frac{Std(close, d)}{close}
$$


### **13. BETA(d) – Linear Trend Slope (normalized)**

**Definicja:**
Nachylenie regresji liniowej ceny zamknięcia z ostatnich *d* dni, znormalizowane przez bieżące close.

**Wzór:**
$$
BETA_d = \frac{Slope(close, d)}{close}
$$


### **14. RSQR(d) – R-squared of Linear Trend**

**Definicja:**
Wartość (R^2) regresji liniowej ceny w ostatnich *d* dniach. Mierzy liniowość trendu.

**Wzór:**
$$
RSQR_d = Rsquare(close, d)
$$


### **15. RESI(d) – Regression Residual (normalized)**

**Definicja:**
Odchylenie punktów od linii regresji liniowej (residual), znormalizowane przez close.

**Wzór:**
$$
RESI_d = \frac{Resi(close, d)}{close}
$$


### **16. MAX(d) – Highest High (normalized)**

**Definicja:**
Najwyższa cena high z ostatnich d dni, znormalizowana aktualnym close.

**Wzór:**
$$
MAX_d = \frac{Max(high, d)}{close}
$$

### **17. MIN(d) – Lowest Low (normalized)**

**Definicja:**
Najniższa cena low z ostatnich d dni, znormalizowana aktualnym close.

**Wzór:**
$$
MIN_d = \frac{Min(low, d)}{close}
$$


### **18. QTLU(d) – Upper Quantile (80%) of Close**

**Definicja:**
80-percentyl ceny zamknięcia z ostatnich *d* dni, znormalizowany przez close.

**Wzór:**
$$
QTLU_d = \frac{Quantile(close, d, 0.8)}{close}
$$


### **19. QTLD(d) – Lower Quantile (20%) of Close**

**Definicja:**
20-percentyl ceny zamknięcia z ostatnich *d* dni, znormalizowany przez close.

**Wzór:**
$$
QTLD_d = \frac{Quantile(close, d, 0.2)}{close}
$$


### **20. RANK(d) – Percentile Rank of Close**

**Definicja:**
Pozycja bieżącej ceny względem cen zamknięcia z ostatnich *d* dni (percentyl 0–1).

**Wzór:**
$$
RANK_d = Rank(close, d)
$$


### **21. RSV(d) – Raw Stochastic Value**

**Definicja:**
Pozycja bieżącego close w zakresie między minimum i maksimum z ostatnich *d* dni.

**Wzór:**
$$
RSV_d =
\frac{close - Min(low, d)}
{Max(high, d) - Min(low, d) + 10^{-12}}
$$


### **22. IMAX(d) – Days Since Last Highest High**

**Definicja:**
Liczba dni od ostatniego maksimum high, znormalizowana przez *d*.
Część wskaźnika Aroon Up.

**Wzór:**
$$
IMAX_d = \frac{IdxMax(high, d)}{d}
$$

### **23. IMIN(d) – Days Since Last Lowest Low**

**Definicja:**
Liczba dni od ostatniego minimum low, znormalizowana przez *d*.
Część Aroon Down.

**Wzór:**
$$
IMIN_d = \frac{IdxMin(low, d)}{d}
$$


### **24. IMXD(d) – Time Between High and Low Events**

**Definicja:**
Różnica czasu między wystąpieniem ostatniego high i low w oknie *d*.
Duża wartość sugeruje silny spadkowy momentum.

**Wzór:**
$$
IMXD_d = \frac{IdxMax(high, d) - IdxMin(low, d)}{d}
$$


### **25. CORR(d) – Correlation between Price and Volume**

**Definicja:**
Korelacja pomiędzy ceną close a logarytmem wolumenu z ostatnich *d* dni.

**Wzór:**
$$
CORR_d = Corr(close,\ \log(volume+1),\ d)
$$


### **26. CORD(d) – Correlation of Price Returns and Volume Change**

**Definicja:**
Korelacja pomiędzy dzienną stopą zwrotu a zmianą wolumenu.

**Wzór:**
$$
CORD_d = Corr\left(\frac{close}{Ref(close,1)},\ \log\left(\frac{volume}{Ref(volume,1)}+1\right),\ d\right)
$$


### **27. CNTP(d) – % Up Days**

**Definicja:**
Procent dni, w których cena close wzrosła względem poprzedniego dnia.

**Wzór:**
$$
CNTP_d = Mean(close > Ref(close,1),\ d)
$$


### **28. CNTN(d) – % Down Days**

**Definicja:**
Procent dni, w których close spadło.

**Wzór:**
$$
CNTN_d = Mean(close < Ref(close,1),\ d)
$$


### **29. CNTD(d) – Net Up/Down Ratio**

**Definicja:**
Różnica między procentem dni wzrostowych i spadkowych.

**Wzór:**
$$
CNTD_d = Mean(close>Ref(close,1),d)\ -\ Mean(close<Ref(close,1),d)
$$


### **30. SUMP(d) – Ratio of Total Gains**

**Definicja:**
Udział sumy wzrostów w całkowitej zmianie absolutnej ceny (odpowiednik RSI).

**Wzór:**
$$
SUMP_d =
\frac{
Sum\left(\max(close-Ref(close,1),0), d\right)
}
{
Sum\left(|close-Ref(close,1)|, d\right) + 10^{-12}
}
$$


### **31. SUMN(d) – Ratio of Total Losses**

**Definicja:**
Udział sumy spadków w całkowitej zmianie absolutnej ceny (1 − SUMP).

**Wzór:**
$$
SUMN_d =
\frac{
Sum\left(\max(Ref(close,1)-close,0), d\right)
}
{
Sum(|close-Ref(close,1)|, d) + 10^{-12}
}
$$


### **32. SUMD(d) – Net Gain/Loss Ratio**

**Definicja:**
Różnica udziału wzrostów i spadków, w stylu wskaźników RSI.

**Wzór:**
$$
SUMD_d =
\frac{
Sum(\max(close-Ref(close,1),0),d)
-

Sum(\max(Ref(close,1)-close,0),d)
}
{
Sum(|close-Ref(close,1)|,d) + 10^{-12}
}
$$


### **33. VMA(d) – Volume Moving Average (normalized)**

**Definicja:**
Średnia wolumenu z ostatnich *d* dni, podzielona przez bieżący wolumen.

**Wzór:**
$$
VMA_d = \frac{Mean(volume, d)}{volume + 10^{-12}}
$$


### **34. VSTD(d) – Volume Standard Deviation (normalized)**

**Definicja:**
Odchylenie standardowe wolumenu z ostatnich *d* dni, znormalizowane przez wolumen.

**Wzór:**
$$
VSTD_d = \frac{Std(volume, d)}{volume + 10^{-12}}
$$


### **35. WVMA(d) – Volume-Weighted Price Volatility**

**Definicja:**
Odchylenie standardowe wolumenowo-ważonej zmienności ceny.

**Wzór:**
$$
WVMA_d =
\frac{
Std(|\frac{close}{Ref(close,1)} - 1| \cdot volume, d)
}
{
Mean(|\frac{close}{Ref(close,1)} - 1| \cdot volume, d) + 10^{-12}
}
$$


### **36. VSUMP(d) – Ratio of Volume Gains**

**Definicja:**
Udział dni ze wzrostem wolumenu.

**Wzór:**
$$
VSUMP_d =
\frac{
Sum(\max(volume-Ref(volume,1),0),d)
}
{
Sum(|volume - Ref(volume,1)|, d) + 10^{-12}
}
$$


### **37. VSUMN(d) – Ratio of Volume Losses**

**Definicja:**
Udział dni ze spadkiem wolumenu (1 − VSUMP).

**Wzór:**
$$
VSUMN_d =
\frac{
Sum(\max(Ref(volume,1)-volume,0),d)
}
{
Sum(|volume - Ref(volume,1)|, d) + 10^{-12}
}
$$


### **38. VSUMD(d) – Net Volume Gain/Loss Ratio**

**Definicja:**
Różnica wolumenowych wzrostów i spadków, analogicznie do RSI dla wolumenu.

**Wzór:**
$$
VSUMD_d =
\frac{
Sum(\max(volume-Ref(volume,1),0),d)
-
Sum(\max(Ref(volume,1)-volume,0),d)
}
{
Sum(|volume-Ref(volume,1)|, d) + 10^{-12}
}
$$




In [73]:
import numpy as np

def calculate_alpha158(df):
    df = df.copy()
    df["KMID"] = (df["close"] - df["open"])/df["open"]
    df["KLEN"] = (df["high"] - df["low"])/df["open"]
    df["KMID2"] = (df["close"]-df["open"])/(df["high"]-df["low"]+1e-12)
    df["KUP"] = (df["high"] - df[["open", "close"]].max(axis=1))/df["open"]
    df["KUP2"] = (df["high"] - df[["open", "close"]].max(axis=1))/(df["high"]-df["low"]+1e-12)
    df["KLOW"] = (df[["open", "close"]].min(axis=1) - df["low"])/df["open"]
    df["KLOW2"] = (df[["open", "close"]].min(axis=1) - df["low"])/(df["high"]-df["low"]+1e-12)
    df["KSFT"] = (2*df["close"] - df["high"] - df["low"])/df["open"]
    df["KSFT2"] = (2*df["close"] - df["high"] - df["low"])/(df["high"]-df["low"]+1e-12)
    windows = [5, 10, 20, 30, 60]
    for i in windows:
        df[f"ROC{i}"] = df["close"].shift(i)/df["close"]
        df[f"MA{i}"] = df["close"].rolling(i).mean()/df["close"]
        df[f"STD{i}"] = df["close"].rolling(i).std()/df["close"]

        def slope(y):
            x = np.arange(len(y))
            return np.polyfit(x, y, 1)[0]

        df[f"BETA{i}"] = df["close"].rolling(i).apply(slope)/df["close"]

        def rsquare(y):
            x = np.arange(len(y))
            slope, intercept = np.polyfit(x, y, 1)
            y_pred = slope * x + intercept
            ss_total = np.sum((y - np.mean(y))**2)
            ss_residual = np.sum((y - y_pred)**2)
            return 1 - (ss_residual / ss_total) if ss_total != 0 else 0
        
        df[f"RSQR{i}"] = df["close"].rolling(i).apply(rsquare)
        def residual(y):
            x = np.arange(len(y))
            slope, intercept = np.polyfit(x, y, 1)
            y_pred = slope * x + intercept
            return y.iloc[-1] - y_pred[-1]
        df[f"RESI{i}"] = df["close"].rolling(i).apply(residual)/df["close"]
        df[f"MAX{i}"] = df["high"].rolling(i).max()/df["close"]
        df[f"MIN{i}"] = df["low"].rolling(i).min()/df["close"]
        df[f"QTLU{i}"] = df["close"].rolling(i).quantile(0.8)/df["close"]
        df[f"QTLD{i}"] = df["close"].rolling(i).quantile(0.2)/df["close"]
        def rank_percentile(y):
            return y.rank().iloc[-1]/len(y)
        df[f"RANK{i}"] = df["close"].rolling(i).apply(rank_percentile)
        df[f"RSV{i}"] = (df["close"] - df["low"].rolling(i).min())/(df["high"].rolling(i).max() - df["low"].rolling(i).min()+1e-12)
        def idxmax(y):
            idx = np.where(y == y.max())[0][-1]
            return len(y) - idx - 1
        df[f"IMAX{i}"] = df["high"].rolling(i).apply(idxmax)/i
        def idxmin(y):
            idx = np.where(y == y.min())[0][-1]
            return len(y) - idx - 1
        df[f"IMIN{i}"] = df["low"].rolling(i).apply(idxmin)/i
        df[f"IMXD{i}"] = (df["high"].rolling(i).apply(idxmax) - df["low"].rolling(i).apply(idxmin))/i
        df[f"CORR{i}"] = df["close"].rolling(i).corr(np.log1p(df["volume"]))
        df[f"CORD{i}"] = (df["close"]/df["close"].shift(1)).rolling(i).corr(np.log1p((df["volume"]/df["volume"].shift(1))))
        df[f"CNTP{i}"] = (df["close"] > df["close"].shift(1)).rolling(i).sum()/i
        df[f"CNTN{i}"] = (df["close"] < df["close"].shift(1)).rolling(i).sum()/i
        df[f"CNTD{i}"] = df[f"CNTP{i}"] - df[f"CNTN{i}"]
        df[f"SUMP{i}"] = (df["close"] - df["close"].shift(1)>0).rolling(i).sum()/((df["close"]-df["close"].shift(1)).abs().rolling(i).sum()+1e-12)
        df[f"SUMN{i}"] = (df["close"] - df["close"].shift(1)<0).rolling(i).sum()/((df["close"]-df["close"].shift(1)).abs().rolling(i).sum()+1e-12)
        df[f"SUMD{i}"] = ((df["close"] - df["close"].shift(1)>0).rolling(i).sum() - (df["close"] - df["close"].shift(1)<0).rolling(i).sum())/ ((df["close"]-df["close"].shift(1)).abs().rolling(i).sum()+1e-12)
        df[f"VMA{i}"] = df["volume"].rolling(i).mean()/(df["volume"]+1e-12)
        df[f"VSTD{i}"] = df["volume"].rolling(i).std()/(df["volume"]+1e-12)
        df[f"WVMA{i}"] = (df["close"]/df["close"].shift(1)-1).abs().rolling(i).mean()/(((df["close"]-df["close"].shift(1)-1).abs()*df["volume"]).rolling(i).mean()+1e-12)
        df[f"VSUMP{i}"] = (df["volume"]-df["volume"].shift(1)>0).rolling(i).sum()/((df["volume"]-df["volume"].shift(1)).abs().rolling(i).sum()+1e-12)
        df[f"VSUMN{i}"] = (df["volume"]-df["volume"].shift(1)<0).rolling(i).sum()/((df["volume"]-df["volume"].shift(1)).abs().rolling(i).sum()+1e-12)
        df[f"VSUMD{i}"] = ((df["volume"]-df["volume"].shift(1)>0).rolling(i).sum() - (df["volume"]-df["volume"].shift(1)<0).rolling(i).sum())/ ((df["volume"]-df["volume"].shift(1)).abs().rolling(i).sum()+1e-12)


    return df

In [74]:
alpha158 = calculate_alpha158(data)

/var/folders/g3/sxyl_n7x6d51bbb320nhxx5c0000gn/T/ipykernel_7089/588504408.py:24: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[f"BETA{i}"] = df["close"].rolling(i).apply(slope)/df["close"]
/var/folders/g3/sxyl_n7x6d51bbb320nhxx5c0000gn/T/ipykernel_7089/588504408.py:34: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[f"RSQR{i}"] = df["close"].rolling(i).apply(rsquare)
/var/folders/g3/sxyl_n7x6d51bbb320nhxx5c0000gn/T/ipykernel_7089/588504408.py:40: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result

In [75]:
alpha360 = calculate_alpha360(data)

/var/folders/g3/sxyl_n7x6d51bbb320nhxx5c0000gn/T/ipykernel_7089/3058867393.py:7: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[f"open{i}"] = df["open"].shift(i)/df["close"]
/var/folders/g3/sxyl_n7x6d51bbb320nhxx5c0000gn/T/ipykernel_7089/3058867393.py:8: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[f"volume{i}"] = df["volume"].shift(i)/(df["volume"]+1e-12)
/var/folders/g3/sxyl_n7x6d51bbb320nhxx5c0000gn/T/ipykernel_7089/3058867393.py:9: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calli